In [50]:
import openai
from openai import OpenAI
import pandas as pd
from constants import PROMPT_SANITY_CHECK, OPENAI_KEY
from utils import encode_image
import re
import string

In [2]:
df = pd.read_csv('data/final_data_descriptions.csv')

In [3]:
df

,Unnamed: 0.1,Unnamed: 0,image_path,image_id,question,answer,Tm,Tc,Ans,Ti
0,0,0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red,"Several days after the low dissipated , the r..."
1,1,1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,red,Kawaguchi 's Center Body of troops was planni...
2,2,2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red,"In the centre , the main attack along the Bui..."
3,3,3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red,Mount Elbert was named by miners in honor of ...
4,4,4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,red,"Runs west through Jackson , Mississippi , eve..."
...,...,...,...,...,...,...,...,...,...,...
991,991,991,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no,The documentary film Tim Richmond : To The Li...
992,992,992,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics.",Kevin Spacey as David <unk> \n
993,993,993,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,blue,The Island Def Jam rapper Big K.R.I.T. was bo...
994,994,994,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red,"A total of 2 @,@ 000 people attended Slammive..."


In [4]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

In [5]:
model = OpenAI(api_key=OPENAI_KEY)

In [6]:
def sanity_check_per_text(text_col: str):

    answers_sanity_check = []

    for i in df.index:
        image_infos = df.iloc[i]
        image_description = image_infos[text_col]
        image_question = image_infos['question']

        prompt_formatado = PROMPT_SANITY_CHECK.format(text=image_description, question=image_question)

        print(f"Gerando resposta para a imagem {i}.")

        response = model.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_formatado
                        }
                    ]
                }
            ], temperature=0.0, timeout=20.0
        )
        texts = response.choices[0].message.content
        answers_sanity_check.append(texts)

        print(f"Textos gerados: {texts}")

    return answers_sanity_check

In [7]:
sanity_check_tc = sanity_check_per_text('Tc')

Gerando resposta para a imagem 0.
Textos gerados: Red
Gerando resposta para a imagem 1.
Textos gerados: AB-1234
Gerando resposta para a imagem 2.
Textos gerados: Yes.
Gerando resposta para a imagem 3.
Textos gerados: Red
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: Reading a book.
Gerando resposta para a imagem 6.
Textos gerados: Red
Gerando resposta para a imagem 7.
Textos gerados: Three.
Gerando resposta para a imagem 8.
Textos gerados: Soccer
Gerando resposta para a imagem 9.
Textos gerados: Lemons
Gerando resposta para a imagem 10.
Textos gerados: Turn the red handle.
Gerando resposta para a imagem 11.
Textos gerados: A blue chair.
Gerando resposta para a imagem 12.
Textos gerados: Green.
Gerando resposta para a imagem 13.
Textos gerados: Sunset
Gerando resposta para a imagem 14.
Textos gerados: No.
Gerando resposta para a imagem 15.
Textos gerados: No.
Gerando resposta para a imagem 16.
Textos gerados: A metal chair.
Geran

In [9]:
sanity_check_tm = sanity_check_per_text('Tm')

Gerando resposta para a imagem 0.
Textos gerados: Blue
Gerando resposta para a imagem 1.
Textos gerados: SV-6260
Gerando resposta para a imagem 2.
Textos gerados: Yes.
Gerando resposta para a imagem 3.
Textos gerados: Blue
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: Looking outside.
Gerando resposta para a imagem 6.
Textos gerados: Yellow
Gerando resposta para a imagem 7.
Textos gerados: One.
Gerando resposta para a imagem 8.
Textos gerados: Baseball
Gerando resposta para a imagem 9.
Textos gerados: Corn
Gerando resposta para a imagem 10.
Textos gerados: Turn the blue handle.
Gerando resposta para a imagem 11.
Textos gerados: Couch
Gerando resposta para a imagem 12.
Textos gerados: Red
Gerando resposta para a imagem 13.
Textos gerados: Cupcake
Gerando resposta para a imagem 14.
Textos gerados: Yes.
Gerando resposta para a imagem 15.
Textos gerados: Yes.
Gerando resposta para a imagem 16.
Textos gerados: A wooden bench.
Gerando

In [ ]:
sanity_check_ti = sanity_check_per_text('Ti')

Gerando resposta para a imagem 0.
Textos gerados: The text does not provide information about the color of the gym shoes.
Gerando resposta para a imagem 1.
Textos gerados: The text does not provide a license number.
Gerando resposta para a imagem 2.
Textos gerados: The text does not provide information about a dog or a collar.
Gerando resposta para a imagem 3.
Textos gerados: The text does not provide information about the color of a lamp.
Gerando resposta para a imagem 4.
Textos gerados: No.
Gerando resposta para a imagem 5.
Textos gerados: The text does not provide information about a woman in the room.
Gerando resposta para a imagem 6.
Textos gerados: The text does not provide information about the color of the birds' tails.
Gerando resposta para a imagem 7.
Textos gerados: The text does not provide information about paper towel rolls.
Gerando resposta para a imagem 8.
Textos gerados: The text does not specify a sport being played.
Gerando resposta para a imagem 9.
Textos gerados: C

In [ ]:
sanity_check_gpt = pd.DataFrame({'responses_tc': sanity_check_tc, 'responses_tm': sanity_check_tm, 'responses_ti': sanity_check_ti})

In [22]:
sanity_check_gpt

,responses_tc,responses_tm,responses_ti
0,Red,Blue,The text does not provide information about th...
1,AB-1234,SV-6260,The text does not provide a license number.
2,Yes.,Yes.,The text does not provide information about a ...
3,Red,Blue,The text does not provide information about th...
4,No.,No.,No.
...,...,...,...
991,Unknown,Yes.,Unknown
992,Unknown,Yes.,No.
993,Yes.,No.,The text does not provide information about a ...
994,A red object.,A blue object.,The text does not provide information about an...


In [25]:
base_answers = []
base_image_path = 'images/'

for i in range(df.shape[0]):

    image_infos = df.iloc[i]
    image_question = image_infos['question']
    image_path = base_image_path + image_infos['image_path']
    
    encoded_image = encode_image(image_path)

    print(f"Gerando textos para a imagem {i}.")

    response = model.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url" : {"url": f"data:image/jpeg;base64,{encoded_image}"}
                    },
                    {
                        "type": "text",
                        "text": f"{image_question} Respond only with the final answer, without explanation or full sentences."
                    }
                ]
            }
        ], temperature=0.0, timeout=20.0
    )
    texts = response.choices[0].message.content
    base_answers.append(texts)

    print(f"Textos gerados: {texts}")


Gerando textos para a imagem 0.
Textos gerados: White
Gerando textos para a imagem 1.
Textos gerados: SV-6260
Gerando textos para a imagem 2.
Textos gerados: Yes.
Gerando textos para a imagem 3.
Textos gerados: Blue
Gerando textos para a imagem 4.
Textos gerados: No.
Gerando textos para a imagem 5.
Textos gerados: Looking out the window.
Gerando textos para a imagem 6.
Textos gerados: Yellow
Gerando textos para a imagem 7.
Textos gerados: One.
Gerando textos para a imagem 8.
Textos gerados: Baseball
Gerando textos para a imagem 9.
Textos gerados: Corn
Gerando textos para a imagem 10.
Textos gerados: Turn the faucet handle to the right.
Gerando textos para a imagem 11.
Textos gerados: He is standing on a Wii Balance Board.
Gerando textos para a imagem 12.
Textos gerados: Red
Gerando textos para a imagem 13.
Textos gerados: Cupcake
Gerando textos para a imagem 14.
Textos gerados: Yes.
Gerando textos para a imagem 15.
Textos gerados: Yes.
Gerando textos para a imagem 16.
Textos gerados: B

In [26]:
sanity_check_gpt['base_response'] = base_answers

In [44]:
sanity_check_gpt['real_response'] = df['answer']

In [49]:
sanity_check_gpt

,responses_tc,responses_tm,responses_ti,base_response,real_response
0,Red,Blue,The text does not provide information about th...,White,white
1,AB-1234,SV-6260,The text does not provide a license number.,SV-6260,sv-6260
2,Yes.,Yes.,The text does not provide information about a ...,Yes.,no
3,Red,Blue,The text does not provide information about th...,Blue,blue
4,No.,No.,No.,No.,no
...,...,...,...,...,...
991,Unknown,Yes.,Unknown,Yes.,yes
992,Unknown,Yes.,No.,Yes.,yes
993,Yes.,No.,The text does not provide information about a ...,No.,no
994,A red object.,A blue object.,The text does not provide information about an...,A red object.,nothing


In [54]:
sanity_check_gpt.to_csv('data/sanity_check.csv')

In [ ]:
NUMBER_MAP = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9",
    "ten": "10"
}

CONTRACTIONS = {
    "dont": "don't", "doesnt": "doesn't", "cant": "can't",
    "couldnt": "couldn't", "shouldnt": "shouldn't",
    "wouldnt": "wouldn't", "isnt": "isn't", "arent": "aren't",
    "wasnt": "wasn't", "werent": "weren't",
    "im": "i'm", "ive": "i've", "ill": "i'll",
    "youre": "you're", "youve": "you've", "youll": "you'll",
    "theyre": "they're", "weve": "we've", "were": "we're"
}

ARTICLES = {"a", "an", "the"}

def normalize_answer_vqa(ans: str) -> str:
    if not isinstance(ans, str):
        return ans

    # 1. lowercase
    ans = ans.lower().strip()

    # 2. remove periods that are NOT decimal separators
    ans = re.sub(r'(?<!\d)\.(?!\d)', '', ans)

    # 3. convert number words → digits
    ans = " ".join(NUMBER_MAP.get(w, w) for w in ans.split())

    # 4. remove articles
    ans = " ".join(w for w in ans.split() if w not in ARTICLES)

    # 5. expand contractions
    for wrong, right in CONTRACTIONS.items():
        ans = ans.replace(wrong, right)

    # 6. remove commas NOT between digits (keep 100,978)
    ans = re.sub(r'(?<!\d),(?!\d)', ' ', ans)

    # 7. replace punctuation except apostrophe, colon, comma, and dot
    # (dot must be preserved to keep decimals)
    keep = "' :.,"
    to_remove = ''.join(ch for ch in string.punctuation if ch not in keep)
    ans = re.sub(f"[{re.escape(to_remove)}]", " ", ans)

    # 8. compress spaces
    ans = re.sub(r"\s+", " ", ans).strip()

    return ans


In [48]:
# testando normalização de texto

t = normalize_answer_vqa('bread A the an TEN 10.0 10,0. A red object. One apple, please!')
t

'bread 10 10.0 10,0. red object 1 apple please'

In [63]:
def evaluate_response(generated_responses: list, real_responses: list):

    matches = 0
    accuracy = None

    if len(generated_responses) != len(real_responses):
        print('Listas diferem em tamanho.')
        return accuracy
    
    else:
        for i, generated_response in enumerate(generated_responses):
            normalized_generated_response = normalize_answer_vqa(generated_response)
            normalized_real_response = normalize_answer_vqa(real_responses[i])
            print(f"Comparando {normalized_generated_response} e {normalized_real_response}")
            if normalized_generated_response == normalized_real_response:
                matches += 1

    accuracy = matches/len(generated_responses)

    return accuracy

In [68]:
# calculando acurácias para sanity check

real_responses = list(df['answer'])
sanity_check_tm = list(sanity_check_gpt['responses_tm'])
sanity_check_tc = list(sanity_check_gpt['responses_tc'])
sanity_check_ti = list(sanity_check_gpt['responses_ti'])

accuracy_base = evaluate_response(base_answers, real_responses)
accuracy_tm = evaluate_response(sanity_check_tm, real_responses)
accuracy_tc = evaluate_response(sanity_check_tc, real_responses)
accuracy_ti = evaluate_response(sanity_check_ti, real_responses)

Comparando white e white
Comparando sv 6260 e sv 6260
Comparando yes e no
Comparando blue e blue
Comparando no e no
Comparando looking out window e cleaning
Comparando yellow e yellow
Comparando 1 e 1
Comparando baseball e baseball
Comparando corn e corn
Comparando turn faucet handle to right e with handle
Comparando he is standing on wii balance board e chair
Comparando red e red
Comparando cupcake e cupcake
Comparando yes e yes
Comparando yes e yes
Comparando bench e bench
Comparando no e no
Comparando i don't know e umpires
Comparando overcast e overcast
Comparando yes e yes
Comparando on top of red laptop e under phone
Comparando orange e orange
Comparando basil e basil
Comparando ms 815 aj e e 820 cr ms 815 aj
Comparando yes e yes
Comparando polka dot tie e tie
Comparando camera e camera
Comparando yes e yes
Comparando baseball glove e glove
Comparando no e no
Comparando yes e yes
Comparando i don't know e no
Comparando boat e water
Comparando living room e living room
Comparando 

In [71]:
print(f"Acurácia Base: {accuracy_base}\n Acurácia Tm: {accuracy_tm}\n Acurácia Tc: {accuracy_tc}\n Acurácia Ti: {accuracy_ti}")

Acurácia Base: 0.6124497991967871
 Acurácia Tm: 0.536144578313253
 Acurácia Tc: 0.08032128514056225
 Acurácia Ti: 0.07228915662650602


In [70]:
accuracy_tm = evaluate_response(sanity_check_tm, real_responses)

Comparando blue e white
Comparando sv 6260 e sv 6260
Comparando yes e no
Comparando blue e blue
Comparando no e no
Comparando looking outside e cleaning
Comparando yellow e yellow
Comparando 1 e 1
Comparando baseball e baseball
Comparando corn e corn
Comparando turn blue handle e with handle
Comparando couch e chair
Comparando red e red
Comparando cupcake e cupcake
Comparando yes e yes
Comparando yes e yes
Comparando wooden bench e bench
Comparando no e no
Comparando umpires e umpires
Comparando overcast e overcast
Comparando yes e yes
Comparando on top of red laptop e under phone
Comparando orange e orange
Comparando basil leaves e basil
Comparando ms 815 aj e e 820 cr ms 815 aj
Comparando yes e yes
Comparando brown polka dot tie e tie
Comparando camera e camera
Comparando yes e yes
Comparando baseball glove e glove
Comparando no e no
Comparando yes e yes
Comparando unknown e no
Comparando there is no bridge archway present e water
Comparando waiting area e living room
Comparando 4 e 

In [72]:
sanity_check_gpt.loc[sanity_check_gpt['real_response'] == '6']

,responses_tc,responses_tm,responses_ti,base_response,real_response
84,Twelve birds.,Six birds.,The text does not provide information about bi...,Six birds.,6
88,Several apples.,Several bananas.,The text does not provide information about fr...,Six.,6
138,The text does not provide information on the n...,Four tires.,The text does not provide information about a ...,Six.,6
234,The text does not specify the number of people.,The text does not specify the number of people.,The text does not provide information about th...,Seven,6
241,Three.,Six.,The text does not provide information about pe...,Seven,6
527,Three.,Two.,The text does not provide information about th...,Two.,6
827,Two.,Four.,Four.,Six.,6
892,One,One.,The text does not provide information about th...,Five.,6


In [74]:
df.loc[df['answer'] == '6']

,image_path,image_id,question,answer,Tm,Tc,Ans,Ti
84,COCO_val2014_000000001103.jpg,1103,Is it six or twelve birds?,6,The image shows six birds running along the sh...,The image shows twelve birds running along the...,twelve,Richmond excelled in sports ; he set a confer...
88,COCO_val2014_000000001153.jpg,1153,How many fruits are on this plate?,6,"A plate with several bananas, some of which ar...","A plate with several apples, some of which are...",red,"TNA held two $ 25 @,@ 000 Fan Challenge segme..."
138,COCO_val2014_000000001739.jpg,1739,How many tires are on this truck?,6,The image shows an old truck with a tarp cover...,The image shows an old truck with a tarp cover...,blue,"The site , however , was often at war and ins..."
234,COCO_val2014_000000003255.jpg,3255,What number of people are standing in front of...,6,A group of people is standing in front of a sn...,A group of people is standing in front of a sa...,yellow,The Bat Palace is also known as the Palace of...
241,COCO_val2014_000000003382.jpg,3382,How many people can be seen?,6,The image shows six people in a naval control ...,The image shows three people in a naval contro...,red,"The planet is unlikely to have large moons , ..."
527,COCO_val2014_000000007414.jpg,7414,How many vehicles on the road?,6,"A road is blocked by a large flock of sheep, w...","A road is blocked by a large flock of sheep, w...",green,Richard Button ( Jr ) 1860 @-@ 62 \n
827,COCO_val2014_000000011615.jpg,11615,How many signpost are seen?,6,The image shows four directional signposts sta...,The image shows two directional signposts stac...,red,"By the Late Classic , a network of <unk> ( ca..."
892,COCO_val2014_000000012670.jpg,12670,How many scarves are there?,6,"The image shows a crowd of people, with one pe...","The image shows a crowd of people, with one pe...",red,"The tour was recorded in VHS format , and was..."


In [75]:
df.iloc[892]['Tc']

'The image shows a crowd of people, with one person wearing a red scarf prominently visible in the foreground.'